<a href="https://colab.research.google.com/github/forest1988/colaboratory/blob/main/prophetnet_seq2seqtrainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

In [2]:
torch.__version__

'1.7.0+cu101'

In [3]:
!git clone https://github.com/huggingface/transformers.git
#git clone https://github.com/forest1988/transformers.git

Cloning into 'transformers'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 51115 (delta 3), reused 5 (delta 0), pack-reused 51094
Receiving objects: 100% (51115/51115), 37.52 MiB | 27.36 MiB/s, done.
Resolving deltas: 100% (35699/35699), done.


In [4]:
%cd transformers/

/content/transformers


In [5]:
!pip install -e .

Obtaining file:///content/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 2.9MB 12.2MB/s 
     |████████████████████████████████| 890kB 47.8MB/s 
     |████████████████████████████████| 1.1MB 46.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=02b38251fd3d4281477f42842fd9ea990b2b85cba16a6a870e0f3b72258e46d7
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
  Running setup.py develop for transformers


In [6]:
%cd /content/transformers/examples/seq2seq/

/content/transformers/examples/seq2seq


In [7]:
# to avoid the "ModuleNotFoundError" 
import sys
print(sys.path)
sys.path.append('/content/transformers/src')
print(sys.path)

['', '/env/python', '/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '/usr/local/lib/python3.6/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.6/dist-packages/IPython/extensions', '/root/.ipython']
['', '/env/python', '/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '/usr/local/lib/python3.6/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.6/dist-packages/IPython/extensions', '/root/.ipython', '/content/transformers/src']


In [8]:
# If "ModuleNotFoundError" occurs in this cell, please re-run the runtime. 
# (The first time Colab runtime runs, it seems the sys.path is not correctly updated when we use `pip install -e .`)

import transformers

In [9]:
transformers.__version__

'3.5.0'

## dataset 

In [10]:
!wget https://cdn-datasets.huggingface.co/summarization/xsum.tar.gz
!tar -xzvf xsum.tar.gz
# !export XSUM_DIR=${PWD}/xsum
# %env XSUM_DIR=${PWD}/xsum

--2020-11-13 08:22:17--  https://cdn-datasets.huggingface.co/summarization/xsum.tar.gz
Resolving cdn-datasets.huggingface.co (cdn-datasets.huggingface.co)... 99.84.178.19, 99.84.178.4, 99.84.178.12, ...
Connecting to cdn-datasets.huggingface.co (cdn-datasets.huggingface.co)|99.84.178.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 204844092 (195M) [application/x-tar]
Saving to: ‘xsum.tar.gz’

xsum.tar.gz         100%[===================>] 195.35M  68.3MB/s    in 2.9s    

2020-11-13 08:22:21 (68.3 MB/s) - ‘xsum.tar.gz’ saved [204844092/204844092]

xsum/
xsum/train.target
xsum/train.source
xsum/val.source
xsum/val.target
xsum/test.source
xsum/test.target


In [11]:
import os
os.environ['XSUM_DIR']=os.path.join(os.getcwd(), 'xsum')

In [12]:
!echo $XSUM_DIR

/content/transformers/examples/seq2seq/xsum


In [13]:
!ls $XSUM_DIR

test.source  test.target  train.source	train.target  val.source  val.target


### Run BART-base

In [14]:
%%capture
!pip install gitpython
!pip install rouge_score
!pip install sacrebleu

In [15]:
!python finetune_trainer.py \
    --learning_rate=3e-5 \
    --do_train --do_eval --evaluate_during_training \
    --predict_with_generate \
    --n_train 20 \
    --n_val 10 \
    --model_name_or_path facebook/bart-base \
    --data_dir $XSUM_DIR \
    --output_dir bart-base-tmp \
    --overwrite_output_dir 

2020-11-13 08:22:36.316189: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
/content/transformers/src/transformers/training_args.py:347: FutureWarning: The `evaluate_during_training` argument is deprecated in favor of `evaluation_strategy` (which has more options)
  FutureWarning,
11/13/2020 08:22:39 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
11/13/2020 08:22:39 - INFO - __main__ -   Training/evaluation parameters Seq2SeqTrainingArguments(output_dir='bart-base-tmp', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluate_during_training=True, evaluation_strategy=<EvaluationStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learnin

In [16]:
!nvidia-smi

Fri Nov 13 08:23:37 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P0    46W /  70W |     10MiB / 15079MiB |     49%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Run with ProphetNet (Error occurs) 

In [17]:
from transformers import ProphetNetTokenizer, ProphetNetForConditionalGeneration #, ProphetNetForCausalLM

# --- commeented out to avoid CUDA memory error --- 

# tokenizer = ProphetNetTokenizer.from_pretrained('microsoft/prophetnet-large-uncased')
# model = ProphetNetForConditionalGeneration.from_pretrained('microsoft/prophetnet-large-uncased')

# input_ids = tokenizer("Studies have been shown that owning a dog is good for you", return_tensors="pt").input_ids  # Batch size 1
# decoder_input_ids = tokenizer("Studies show that", return_tensors="pt").input_ids  # Batch size 1
# outputs = model(input_ids=input_ids, decoder_input_ids=decoder_input_ids, return_dict=True)

# logits_next_token = outputs.logits  # logits to predict next token as usual
# logits_ngram_next_tokens = outputs.logits_ngram  # logits to predict 2nd, 3rd, ... next tokens

In [18]:
# test how tokeinzer.decoder works 

# print(tokenizer.decode(input_ids[0]))
# print(tokenizer.decode(input_ids[0], skip_special_tokens=True))

In [19]:
# show the style of the output

# print(type(outputs))
# outputs.keys()

In [20]:
# outputs.logits.shape, outputs.logits_ngram.shape

In [21]:
# out_generate = model.generate(input_ids, num_beams=4, max_length=20, early_stopping=True)

In [22]:
# out_generate.shape, out_generate[0]

In [23]:
# tokenizer.decode(out_generate.shape[0], skip_special_tokens=True)

In [24]:
# --- NotImplementedError occurs here --- 

!python finetune_trainer.py \
    --learning_rate=3e-5 \
    --do_train --do_eval --evaluate_during_training \
    --predict_with_generate \
    --n_train 100 \
    --n_val 10 \
    --model_name_or_path microsoft/prophetnet-large-uncased \
    --data_dir $XSUM_DIR \
    --output_dir tmp \
    --overwrite_output_dir

2020-11-13 08:23:38.862478: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
/content/transformers/src/transformers/training_args.py:347: FutureWarning: The `evaluate_during_training` argument is deprecated in favor of `evaluation_strategy` (which has more options)
  FutureWarning,
11/13/2020 08:23:41 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
11/13/2020 08:23:41 - INFO - __main__ -   Training/evaluation parameters Seq2SeqTrainingArguments(output_dir='tmp', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluate_during_training=True, evaluation_strategy=<EvaluationStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=3e-

### Refer to T5Tokenizer

``` python
    @add_start_docstrings(PREPARE_SEQ2SEQ_BATCH_DOCSTRING)
    def prepare_seq2seq_batch(
        self,
        src_texts: List[str],
        tgt_texts: Optional[List[str]] = None,
        max_length: Optional[int] = None,
        max_target_length: Optional[int] = None,
        padding: str = "longest",
        return_tensors: str = None,
        truncation: bool = True,
        **kwargs,
    ) -> BatchEncoding:
        if max_length is None:
            max_length = self.max_len
        model_inputs = self(
            src_texts,
            add_special_tokens=True,
            return_tensors=return_tensors,
            max_length=max_length,
            padding=padding,
            truncation=truncation,
            **kwargs,
        )
        if tgt_texts is None:
            return model_inputs
        # Process tgt_texts
        if max_target_length is None:
            max_target_length = max_length
        labels_and_decoder_mask = self(
            tgt_texts,
            add_special_tokens=True,
            return_tensors=return_tensors,
            padding=padding,
            max_length=max_target_length,
            truncation=truncation,
            **kwargs,
        )
        model_inputs["labels"] = labels_and_decoder_mask["input_ids"]
        return model_inputs
```

## Try modified version of ProphetNet Tokenizer

In [34]:
%cd /content/

/content


In [35]:
!ls

sample_data  transformers  transformers_modified


In [40]:
import shutil

if os.path.exists("transformers_modified"):
  print("The repository is already cloned. Remove and re-clone it.")
  shutil.rmtree("transformers_modified")

In [41]:
!git clone -b forest1988-prophetnet-prepare-seq2seq-batch https://github.com/forest1988/transformers.git transformers_modified

Cloning into 'transformers_modified'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 49984 (delta 4), reused 11 (delta 4), pack-reused 49965
Receiving objects: 100% (49984/49984), 36.84 MiB | 27.26 MiB/s, done.
Resolving deltas: 100% (34887/34887), done.


In [42]:
%cd transformers_modified/

/content/transformers_modified/transformers_modified


In [43]:
!pip install -e . --upgrade

Obtaining file:///content/transformers_modified/transformers_modified
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Using cached https://files.pythonhosted.org/packages/4c/34/b39eb9994bc3c999270b69c9eea40ecc6f0e97991dba28282b9fd32d44ee/tokenizers-0.9.3-cp36-cp36m-manylinux1_x86_64.whl
  Found existing installation: tokenizers 0.9.2
    Uninstalling tokenizers-0.9.2:
      Successfully uninstalled tokenizers-0.9.2
  Found existing installation: transformers 3.4.0
    Can't uninstall 'transformers'. No files were found to uninstall.
  Running setup.py develop for transformers


In [44]:
%cd /content/transformers_modified/examples/seq2seq/

# Use already downloaded XSUM in /content/transformers/examples/seq2seq/xsum/

/content/transformers_modified/examples/seq2seq


### Run on GPU

In [47]:
# For GPU (if runtime is set to TPU, the code below is run on CPU.)
# max_source_lenght 510

!python finetune_trainer.py \
    --learning_rate=3e-5 \
    --do_train --do_eval --evaluate_during_training \
    --max_source_length 510 \
    --per_device_train_batch_size 2 \
    --predict_with_generate \
    --n_train 300 \
    --n_val 100 \
    --model_name_or_path microsoft/prophetnet-large-uncased \
    --data_dir $XSUM_DIR \
    --output_dir tmp_gpu \
    --overwrite_output_dir

2020-11-13 08:31:59.397701: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
/content/transformers_modified/transformers_modified/src/transformers/training_args.py:347: FutureWarning: The `evaluate_during_training` argument is deprecated in favor of `evaluation_strategy` (which has more options)
  FutureWarning,
11/13/2020 08:32:01 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
11/13/2020 08:32:01 - INFO - __main__ -   Training/evaluation parameters Seq2SeqTrainingArguments(output_dir='tmp_gpu', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluate_during_training=True, evaluation_strategy=<EvaluationStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=2, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumul

In [49]:
# For GPU (if runtime is set to TPU, the code below is run on CPU.)
# max_source_lenght 511

!python finetune_trainer.py \
    --learning_rate=3e-5 \
    --do_train --do_eval --evaluate_during_training \
    --max_source_length 511 \
    --per_device_train_batch_size 2 \
    --predict_with_generate \
    --n_train 300 \
    --n_val 100 \
    --model_name_or_path microsoft/prophetnet-large-uncased \
    --data_dir $XSUM_DIR \
    --output_dir tmp_gpu \
    --overwrite_output_dir

2020-11-13 08:42:59.987235: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
/content/transformers_modified/transformers_modified/src/transformers/training_args.py:347: FutureWarning: The `evaluate_during_training` argument is deprecated in favor of `evaluation_strategy` (which has more options)
  FutureWarning,
11/13/2020 08:43:02 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
11/13/2020 08:43:02 - INFO - __main__ -   Training/evaluation parameters Seq2SeqTrainingArguments(output_dir='tmp_gpu', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluate_during_training=True, evaluation_strategy=<EvaluationStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=2, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumul

In [50]:
# For GPU (if runtime is set to TPU, the code below is run on CPU.)
# max_source_lenght 512

!python finetune_trainer.py \
    --learning_rate=3e-5 \
    --do_train --do_eval --evaluate_during_training \
    --max_source_length 512 \
    --per_device_train_batch_size 2 \
    --predict_with_generate \
    --n_train 300 \
    --n_val 100 \
    --model_name_or_path microsoft/prophetnet-large-uncased \
    --data_dir $XSUM_DIR \
    --output_dir tmp_gpu \
    --overwrite_output_dir

2020-11-13 08:57:26.671954: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
/content/transformers_modified/transformers_modified/src/transformers/training_args.py:347: FutureWarning: The `evaluate_during_training` argument is deprecated in favor of `evaluation_strategy` (which has more options)
  FutureWarning,
11/13/2020 08:57:29 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
11/13/2020 08:57:29 - INFO - __main__ -   Training/evaluation parameters Seq2SeqTrainingArguments(output_dir='tmp_gpu', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluate_during_training=True, evaluation_strategy=<EvaluationStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=2, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumul

### Run on TPU

In [ ]:
# install torch_xla with PyTorch
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.7-cp36-cp36m-linux_x86_64.whl

     |████████████████████████████████| 133.6MB 93kB/s 
     |████████████████████████████████| 61kB 2.9MB/s 
  Found existing installation: google-api-python-client 1.7.12
    Uninstalling google-api-python-client-1.7.12:
      Successfully uninstalled google-api-python-client-1.7.12


In [ ]:
# For TPU

!python xla_spawn.py --num_core 1 \
    finetune_trainer.py \
    --learning_rate=3e-5 \
    --do_train --do_eval --evaluate_during_training \
    --max_source_length 200 \
    --per_device_train_batch_size 32 \
    --prediction_loss_only \
    --model_name_or_path microsoft/prophetnet-large-uncased \
    --data_dir $XSUM_DIR \
    --output_dir tmp_tpu \
    --overwrite_output_dir

2020-11-07 07:35:03.804088: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-11-07 07:35:18.033957: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-11-07 07:35:18.035996: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-11-07 07:35:18.036775: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-11-07 07:35:18.044493: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-11-07 07:35:18.195867: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-11-07 07:35:18.206995: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic libr

In [ ]:
%ls tmp_tpu

config.json           pytorch_model.bin        trainer_state.json
eval_results.json     special_tokens_map.json  training_args.bin
prophetnet.tokenizer  tokenizer_config.json


In [ ]:
!ls

all_results.json		run_distributed_eval.py
bertabs				run_eval.py
builtin_trainer			run_eval_search.py
callbacks.py			runs
convert_model_to_fp16.py	save_len_file.py
convert_pl_checkpoint_to_hf.py	save_randomly_initialized_model.py
distillation.py			sentence_splitter.py
distil_marian_enro_teacher.sh	seq2seq_trainer.py
distil_marian_no_teacher.sh	seq2seq_training_args.py
download_wmt.py			test_bash_script.py
dynamic_bs_example.sh		test_calculate_rouge.py
finetune_bart_tiny.sh		test_data
finetune_pegasus_xsum.sh	test_datasets.py
finetune.py			test_finetune_trainer.py
finetune.sh			test_fsmt_bleu_score.py
finetune_t5.sh			test_make_student.py
finetune_trainer.py		test_seq2seq_examples_multi_gpu.py
__init__.py			test_seq2seq_examples.py
make_student.py			test_tatoeba_conversion.py
minify_dataset.py		tmp_gpu
pack_dataset.py			tmp_tpu
precomputed_pseudo_labels.md	train_distilbart_cnn.sh
__pycache__			train_distilbart_xsum.sh
README.md			train_mbart_cc25_enro.sh
romanian_postprocessing.md	utils

In [ ]:
!cat tmp_tpu/eval_results.json

{
    "eval_loss": 2.2827608585357666,
    "epoch": 3.0
}

In [ ]:
!cat tmp_tpu/config.json

{
  "_name_or_path": "microsoft/prophetnet-large-uncased",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "architectures": [
    "ProphetNetForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 102,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_max_position_embeddings": 514,
  "decoder_start_token_id": 102,
  "disable_ngram_loss": false,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_max_position_embeddings": 513,
  "eos_token_id": 102,
  "eps": 0.0,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "length_penalty": 2.0,
  "max_position_embeddings": 512,
  "model_type": "prophetnet",
  "ngram": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "num_buckets